In [1]:
from kusanagi.ghost.algorithms.ExperienceDataset import ExperienceDataset
from functools import partial
import numpy as np

Using cuDNN version 5005 on context None
Mapped name None to device cuda0: GeForce GTX 770 (0000:01:00.0)


In [57]:
import torch

global_dtype = torch.FloatTensor
#global_dtype = torch.cuda.FloatTensor

MATRIX_STRUCTURES = ['general', 'lower', 'upper', 'symmetric']
class Solve(torch.autograd.Function):
    def __init__(self, A_structure='general'):
        self.A_structure = A_structure
        
    def forward(self, A, b):
        if self.A_structure == 'lower':
            x = torch.trtrs(b, A, False)[0]
        elif self.A_structure == 'upper':
            x = torch.trtrs(b, A, True)[0]
        else:
            x = torch.gesv(b, A)[0]
        
        self.save_for_backward(A, b, x)

        return x
    
    def backward(self, output_grads):
        """
        Reverse-mode gradient updates for matrix solve operation c = A \ b.
        Symbolic expression for updates taken from [1]_.
        References
        ----------
        ..[1] M. B. Giles, "An extended collection of matrix derivative results
          for forward and reverse mode automatic differentiation",
          http://eprints.maths.ox.ac.uk/1079/
        """
        A, b, x = self.saved_tensors
        
        grad_x = output_grads
       
        # transpose solve
        if self.A_structure == 'lower':
            grad_b = torch.trtrs(b, A.t(), True)[0]
        elif self.A_structure == 'upper':
            grad_b = torch.trtrs(b, A.t(), False)[0]
        else:
            grad_b = torch.gesv(b, A.t())[0]
            
        # force outer product if vector second input
        grad_A = -grad_b[:,None].mm(x[None,:])\
                 if x.ndimension() == 1\
                 else -grad_b.mm(x.t())
                
        if self.A_structure == 'lower':
            grad_A = torch.tril(grad_A)
        elif self.A_structure == 'upper':
            grad_A = torch.triu(grad_A)
        
        return grad_A, grad_b
    
def solve_lower_triangular(A, b):
    return Solve(A_structure='lower')(A, b)

def solve_upper_triangular(A, b):
    return Solve(A_structure='upper')(A, b)

class Cholesky(torch.autograd.Function):
    def __init__(self, lower=True):
        self.lower = lower
        
    def forward(self, input):
        chol = torch.potrf(input, not self.lower)
        self.save_for_backward(input,chol)
        return chol
    
    def backward(self, output_grads):
        """
        Cholesky decomposition reverse-mode gradient update.
        Symbolic expression for reverse-mode Cholesky gradient taken from [0]_
        References
        ----------
        .. [0] I. Murray, "Differentiation of the Cholesky decomposition",
           http://arxiv.org/abs/1602.07527
        """

        x, chol_x = self.saved_tensors
        dz = output_grads
        
        # TODO deal with nans 
        
        if not self.lower:
            chol_x = chol_x.t()
            dz = dz.t()

        def tril_and_halve_diagonal(mtx):
            """Extracts lower triangle of square matrix and halves diagonal."""
            return torch.tril(mtx) - torch.diag(torch.diag(mtx)/2)
            
        def conjugate_solve_triangular(outer, inner):
            """Computes L^{-T} P L^{-1} for lower-triangular L."""
            return solve_upper_triangular(
                outer.t(), solve_upper_triangular(outer.t(), inner.t()).t())
        
     
        s = conjugate_solve_triangular(
            chol_x, tril_and_halve_diagonal(chol_x.t().mm(dz)))
        
        s2 = s + s.t()
        sdiag = torch.diag(torch.diag(s))
        
        if self.lower:
            grad = torch.tril(s2) - sdiag
        else:
            grad = torch.triu(s2) - sdiag
            
        return grad
    
def cholesky(input, lower=True):
    return Cholesky(lower)(input)

def maha(X,Y,M=None):
    if not M is None:
        XM = X.mm(M)
        YM = Y.mm(M)
        dist = (XM*X).sum(1).repeat(1, Y.size(0))\
             + (YM*Y).sum(1).t().repeat(X.size(0), 1)\
             - 2*XM.mm(Y.t())
    else:
        dist = (X**2).sum(1).repeat(1,Y.size(0))\
             + (Y**2).sum(1).t().repeat(X.size(0),1)\
             - 2*X.mm(Y.t())
    return dist

def SEard(loghyp, X, Y):
    if Y is None:
        Y = X
    n, D = X.size()
    dist = maha(X, Y, torch.diag(torch.exp(-2*loghyp[:D])))
    return torch.exp(2*loghyp[D].expand(dist.size()) - 0.5*dist )

def Noise(loghyp, X, Y=None):
    if Y is None:
        Y = X
    if X is Y:
        ones = torch.autograd.Variable(torch.ones(X.size(0)))
        K = ones*torch.exp(2*loghyp).expand(ones.size())
        return K.diag()
    else:
        return torch.zeros(X.size(0), X.size(0))
    
def Sum(loghyp_list,cov_list, X, Y=None):
    D = len(cov_list)
    K = sum([cov_list[i](loghyp_list[i], X, Y) for i in range(D)])
    return K

def SEard_params(X, Y):
    n, idims = X.size()
    odims = Y.size(1)
    
    loghyp = torch.autograd.Variable(torch.Tensor(odims,idims+1).type(global_dtype),
                                     requires_grad=True)
    loghyp.data[:, :idims] = 0.5*X.std(0).repeat(odims,1)
    loghyp.data[:, idims] = 0.5*Y.std(0)
    
    return loghyp

def Noise_params(X, Y):
    n, idims = X.size()
    odims = Y.size(1)
    
    loghyp = torch.autograd.Variable(torch.Tensor(odims,1), requires_grad=True)
    loghyp.data[:, 0] = 0.1*Y.std(0)
    
    return loghyp

def GP_loss(loghyps, covs, X, Y):
    if not type(covs) is list:
        covs= covs[covs]
        
    n, idims = X.size()
    odims = Y.size(1)
    
    loss = 0
    X_ = torch.autograd.Variable(X)
    loghyps_ = zip(*loghyps)
    
    for i in range(odims):
        K = Sum(loghyps_[i],covs, X_)
        L = cholesky(K)

In [3]:
exp = ExperienceDataset(filename='/media/diskstation/juan/kusanagi/2016_08_11_first_pool_trial/bellyup_experience')
X, Y = exp.get_dynmodel_dataset()
#convert to torch tensors
X, Y = torch.from_numpy(X).type(global_dtype),torch.from_numpy(Y).type(global_dtype)

[2017-06-15 21:54:10.268943] Experience > Loading state from /media/diskstation/juan/kusanagi/2016_08_11_first_pool_trial/bellyup_experience.zip


In [4]:
X_ = torch.autograd.Variable(X)
loghyps = SEard_params(X, Y), Noise_params(X,Y)
K = Sum(zip(*loghyps)[0],[SEard,Noise], X_, X_)

In [5]:
GP_loss(loghyps, [SEard, Noise], X, Y)

In [6]:
cholK = cholesky(K)

In [299]:
S = torch.randn(165,165)
S = S + S.t()
S = torch.autograd.Variable(S, requires_grad=True)
Y0 = torch.autograd.Variable(Y[:,0,None], requires_grad=True)
R = Solve('general')(S, Y0)

In [94]:
test_ = torch.eye(R.size(0))
R.backward(test_, retain_variables=True)

In [87]:
S.grad.t()

Variable containing:
-1.1325e+00  7.2322e-01  6.1566e-01  ...  -1.6265e-01  1.0828e-01 -1.5203e+00
 7.2322e-01 -4.6186e-01 -3.9317e-01  ...   1.0387e-01 -6.9151e-02  9.7087e-01
 6.1566e-01 -3.9317e-01 -3.3469e-01  ...   8.8423e-02 -5.8867e-02  8.2647e-01
                ...                   ⋱                   ...                
-1.6265e-01  1.0387e-01  8.8423e-02  ...  -2.3361e-02  1.5552e-02 -2.1835e-01
 1.0828e-01 -6.9151e-02 -5.8867e-02  ...   1.5552e-02 -1.0354e-02  1.4536e-01
-1.5203e+00  9.7087e-01  8.2647e-01  ...  -2.1835e-01  1.4536e-01 -2.0409e+00
[torch.FloatTensor of size 165x165]

In [318]:
import theano
def alternative_Rop(f, x, u):
    v = theano.tensor.ones_like(f)    # Dummy variable v of same type as f
    g = theano.tensor.Lop(f, x, v)    # Jacobian of f left multiplied by v
    return theano.tensor.Lop(g.flatten(), v, u)

s = theano.tensor.fmatrix('s')
y = theano.tensor.fvector('y')
ymat = theano.tensor.diag(y)
r = theano.tensor.slinalg.Cholesky()(s)
dr1 = theano.tensor.jacobian(r.flatten(),s)
dr2 = theano.tensor.Lop(r, s, theano.tensor.eye(s.shape[0]))
f = theano.function(outputs=r, inputs=[s])
df1 = theano.function(outputs=dr1, inputs=[s])
df2 = theano.function(outputs=dr2, inputs=[s])

In [319]:
df2(np.eye(165).astype('float32')).shape

(165, 165)

In [320]:
df2(S.data.numpy(), Y0.data.numpy()[:,0]).shape

TypeError: Too many parameter passed to theano function

In [283]:
np.allclose(df1(S.data.numpy(), Y0.data.numpy()[:,0]).reshape((165,165,165)) ,
            df2(S.data.numpy(), Y0.data.numpy()[:,0]).reshape((165,165,165)), atol=1e-4, rtol=1e-4)

True

In [128]:
asd = s.type('v')

In [130]:
type(asd)

theano.tensor.var.TensorVariable